# Interactive Layout-to-Graph Pathfinding 🗺️

This Jupyter Notebook converts a 2D layout image (like a floor plan) into a traversability graph and then uses the **A\* algorithm** to find the most efficient path between two points you select.

### How It Works:
1.  **Configuration**: You define the paths to your layout image, its corresponding color legend, and a custom `KEYWORD_COSTS` dictionary. This dictionary determines how "expensive" it is to move through different types of objects (e.g., making 'rugs' cost more to traverse than 'floor').
2.  **Grid Generation**: The notebook reads your image and uses the legend to create a cost grid. Each cell in the grid gets a movement cost based on the color and your `KEYWORD_COSTS`. Obstacles like 'walls' or 'tables' are assigned an infinite cost, making them impassable.
3.  **Graph Creation**: A `networkx` graph is built from the cost grid, where each traversable cell is a node, and edges connect it to its neighbors.
4.  **Interactive Pathfinding**: A plot of the grid will appear, allowing you to click to select your **start** and **goal** nodes.
5.  **A\* and Visualization**: The A\* algorithm calculates the optimal path, which is then drawn over the cost grid and saved as a new image.

To get started, place your `reconstructed.png` and `color_legend.json` files in a folder named `data` in the same directory as this notebook. Then, simply run the cells in order!

### Imports
All necessary libraries are imported here. The `%matplotlib tk` magic command is used to enable interactive plotting windows.

In [2]:
import json
import random
from pathlib import Path
import numpy as np
import networkx as nx
from PIL import Image
import matplotlib.pyplot as plt
from scipy.spatial import cKDTree

# Use the TkAgg backend for interactive matplotlib plots
%matplotlib tk

## 1. Configuration ⚙️
Modify the variables in this section to experiment with different layouts and cost structures.

In [3]:
# --- Main Configuration ---

# 📂 Set the base path for your input files.
# The notebook expects a 'data' folder in the same directory.
BASE_PATH = Path("./test_images")

# 🖼️ The name of your layout image file.
IMAGE_NAME = "reconstructed.png"

# 🎨 The name of your color legend JSON file.
LEGEND_NAME = "color_legend.json"

# 📏 The size of each grid cell in pixels. A smaller size results in a more detailed
# grid and graph, but takes longer to process.
CELL_SIZE = 1

# 💸 Define the cost of traversing different objects.
# Higher values mean the path will try to avoid these areas.
# np.inf makes an object an impassable obstacle.
KEYWORD_COSTS = {
    # Traversable with low cost
    "floor": 1.0,
    "background": 1.0,

    # Traversable with higher cost (path will avoid if possible)
    "lamp": 1.25,
    "bed": 3.0,
    "rug": 5.0,        # Example: making rugs "slower" to cross
    "plant": 10.0,

    # Obstacles (impassable)
    "wall": np.inf,
    "table": np.inf,
    "chair": np.inf,
    "sofa": np.inf,
    "cabinet": np.inf,
    "shelf": np.inf,
    "door": np.inf,
    "window": np.inf,
    "wardrobe": np.inf,
    "desk": np.inf,
    "toilet": np.inf,
    "sink": np.inf,
    "bath": np.inf,
    "appliance": np.inf
}

# --- Construct full paths ---
IMAGE_PATH = BASE_PATH / IMAGE_NAME
LEGEND_PATH = BASE_PATH / LEGEND_NAME

# --- Check for files ---
if not IMAGE_PATH.exists() or not LEGEND_PATH.exists():
    print("❌ Error: Make sure your image and legend files are in the 'data' directory!")
    print(f"   - Looking for: {IMAGE_PATH}")
    print(f"   - Looking for: {LEGEND_PATH}")

## 2. Core Functions 🛠️
These cells contain the functions that power the conversion and pathfinding logic. You don't need to modify them for typical use.

In [4]:
# Helper class for interactive point selection
class PointSelector:
    def __init__(self, ax, grid):
        self.ax = ax
        self.grid = grid
        self.points = []
        self.cid = ax.figure.canvas.mpl_connect('button_press_event', self)

    def __call__(self, event):
        # Ignore clicks outside the plot axes
        if event.inaxes != self.ax:
            return

        # Convert float coordinates to integer grid indices
        x, y = int(round(event.xdata)), int(round(event.ydata))
        rows, cols = self.grid.shape

        # Check if the point is valid (within bounds and on a traversable cell)
        if not (0 <= y < rows and 0 <= x < cols and np.isfinite(self.grid[y, x])):
            print(f"⚠️ Invalid selection at ({y}, {x}). Please select a traversable (non-obstacle) point.")
            return

        node = (y, x)
        self.points.append(node)

        if len(self.points) == 1:
            print(f"📍 Start node selected: {node}")
            self.ax.plot(x, y, 'go', markersize=10, label='Start')
            print("Please click on the grid to select the GOAL node.")
            self.ax.figure.canvas.draw()
        elif len(self.points) == 2:
            print(f"🏁 Goal node selected: {node}")
            self.ax.plot(x, y, 'r*', markersize=12, label='Goal')
            self.ax.figure.canvas.draw()
            # Disconnect the event handler and close the plot to continue
            self.ax.figure.canvas.mpl_disconnect(self.cid)
            # Use a short timer to ensure the last point is drawn before closing
            self.ax.figure.canvas.start_event_loop(timeout=0.5)

def generate_cost_map(legend: dict, keyword_costs: dict) -> dict:
    """
    Methodically generates a cost map based on a dictionary of keyword-cost pairs.
    """
    cost_map = {}
    print("🤖 Generating advanced cost map from keywords...")

    for label in legend:
        lowered_label = label.lower()
        matched_costs = [cost for keyword, cost in keyword_costs.items() if keyword in lowered_label]
        cost_map[label] = max(matched_costs) if matched_costs else np.inf

    print("✅ Advanced cost map generated.")
    return cost_map

def create_traversability_grid(image_path: Path, legend_path: Path, cell_size: int, cost_map: dict) -> np.ndarray:
    """
    Converts a layout image into a cost-based traversability grid using nearest-color matching.
    """
    print(f"▶️ Loading image '{image_path.name}' and legend '{legend_path.name}'...")
    try:
        img = Image.open(image_path).convert('RGB')
        img_array = np.array(img)
        with open(legend_path, 'r', encoding='utf-8') as f:
            legend = json.load(f)
    except FileNotFoundError as e:
        print(f"❌ Error: {e}")
        raise

    height, width, _ = img_array.shape
    legend_labels = list(legend.keys())
    legend_colors = np.array(list(legend.values()))
    cost_lookup = np.array([cost_map.get(label, np.inf) for label in legend_labels])

    print("ℹ️ Building color KD-Tree for fast nearest-neighbor lookup...")
    color_tree = cKDTree(legend_colors)

    print("ℹ️ Mapping pixels to nearest legend color to determine costs...")
    pixels = img_array.reshape(-1, 3)
    _, closest_color_indices = color_tree.query(pixels, k=1)
    high_res_costs = cost_lookup[closest_color_indices].reshape(height, width)

    grid_height, grid_width = height // cell_size, width // cell_size
    print(f"ℹ️ Image dimensions: {width}x{height} pixels.")
    print(f"ℹ️ Grid cell size: {cell_size}x{cell_size} pixels.")
    print(f"ℹ️ Resulting graph dimensions: {grid_width}x{grid_height} nodes.")
    traversability_grid = np.zeros((grid_height, grid_width), dtype=np.float32)

    print("ℹ️ Downsampling high-resolution cost map to final grid...")
    for r in range(grid_height):
        for c in range(grid_width):
            r_start, r_end = r * cell_size, (r + 1) * cell_size
            c_start, c_end = c * cell_size, (c + 1) * cell_size
            cell_block = high_res_costs[r_start:r_end, c_start:c_end]
            traversability_grid[r, c] = np.max(cell_block)

    print("✅ Cost-based traversability grid generated successfully.")
    return traversability_grid

def create_networkx_graph(grid: np.ndarray) -> nx.Graph:
    """
    Converts a cost-based traversability grid into a NetworkX graph.
    """
    G = nx.Graph()
    rows, cols = grid.shape
    
    for r in range(rows):
        for c in range(cols):
            if np.isfinite(grid[r, c]):
                G.add_node((r, c))

    for r, c in G.nodes():
        for dr in [-1, 0, 1]:
            for dc in [-1, 0, 1]:
                if dr == 0 and dc == 0: continue
                neighbor = (r + dr, c + dc)
                if neighbor in G:
                    cost1 = grid[r, c]
                    cost2 = grid[neighbor[0], neighbor[1]]
                    avg_cost = (cost1 + cost2) / 2.0
                    distance = np.sqrt(dr**2 + dc**2)
                    edge_weight = avg_cost * distance
                    G.add_edge((r, c), neighbor, weight=edge_weight)
    
    print(f"✅ NetworkX graph created with {G.number_of_nodes()} nodes and {G.number_of_edges()} edges.")
    return G

def get_interactive_nodes(grid: np.ndarray):
    """
    Displays the grid and allows the user to interactively select start and goal nodes.
    """
    print("\n--- Interactive Node Selection ---")
    print("A plot window has opened. Please click on the grid to select the START node.")
    
    fig, ax = plt.subplots(figsize=(10, 8))
    viz_grid = np.copy(grid)
    if np.any(np.isinf(viz_grid)):
        max_finite_cost = np.max(viz_grid[np.isfinite(viz_grid)], initial=0)
        viz_grid[np.isinf(viz_grid)] = max_finite_cost * 1.5
    
    im = ax.imshow(viz_grid, cmap='viridis', interpolation='nearest')
    ax.set_title("Interactively Select Start and Goal Nodes", fontsize=16)
    cbar = fig.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    cbar.set_label('Movement Cost', rotation=270, labelpad=15)

    selector = PointSelector(ax, grid)
    plt.show(block=True) # This will block execution until the plot is closed

    if len(selector.points) == 2:
        return selector.points[0], selector.points[1]
    else:
        print("\n⚠️ Node selection was cancelled or not completed.")
        return None, None

def run_astar_pathfinding(graph: nx.Graph, start_node: tuple, goal_node: tuple):
    """
    Finds a path between two specified nodes using A*.
    """
    print("\n--- Running A* Pathfinding ---")
    if start_node is None or goal_node is None:
        return None # Return if nodes were not selected

    if start_node not in graph or goal_node not in graph:
        print(f"❌ Error: Start {start_node} or Goal {goal_node} is not a traversable node.")
        return None
    
    print(f"📍 Start Node: {start_node}")
    print(f"🏁 Goal Node:  {goal_node}")

    def heuristic(a, b):
        return np.sqrt((a[0] - b[0])**2 + (a[1] - b[1])**2)

    try:
        path = nx.astar_path(graph, start_node, goal_node, heuristic=heuristic, weight='weight')
        path_length = nx.astar_path_length(graph, start_node, goal_node, heuristic=heuristic, weight='weight')
        print(f"✅ Path found with {len(path)} nodes and a total cost of {path_length:.2f}.")
        return path
    except nx.NetworkXNoPath:
        print("❌ No path could be found between the start and goal nodes.")
        return None

def visualize_results(image_path: Path, grid: np.ndarray, cell_size: int, path: list = None):
    """
    Generates and displays a comparison plot, including the A* path if provided.
    """
    # Use a non-interactive backend for saving the final plot
    plt.switch_backend('Agg')
    
    img = Image.open(image_path)
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))
    fig.suptitle(f"Layout to Graph Conversion & A* Path (Cell Size: {cell_size}px)", fontsize=16)

    ax1.imshow(img); ax1.set_title("Original Layout Image")
    ax1.set_xticks([]); ax1.set_yticks([])

    viz_grid = np.copy(grid)
    if np.any(np.isinf(viz_grid)):
        max_finite_cost = np.max(viz_grid[np.isfinite(viz_grid)], initial=0)
        viz_grid[np.isinf(viz_grid)] = max_finite_cost * 1.5
    im = ax2.imshow(viz_grid, cmap='viridis', interpolation='nearest')
    ax2.set_title("Resulting Cost Grid & A* Path")
    cbar = fig.colorbar(im, ax=ax2, fraction=0.046, pad=0.04)
    cbar.set_label('Movement Cost', rotation=270, labelpad=15)
    
    if path:
        y_coords = [node[0] for node in path]
        x_coords = [node[1] for node in path]
        ax2.plot(x_coords, y_coords, color='red', linewidth=2, marker='o', markersize=4, label='A* Path')
        ax2.plot(path[0][1], path[0][0], 'go', markersize=10, label='Start')
        ax2.plot(path[-1][1], path[-1][0], 'r*', markersize=12, label='Goal')
        ax2.legend()
    
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    output_filename = image_path.stem.replace('_image', '_pathfinding_visualization.png')
    plt.savefig(output_filename)
    print(f"\n💾 Visualization with path saved to '{output_filename}'")
    
    # Display the plot inline in the notebook
    plt.show()

## 3. Execution Workflow 🚀
Run the following cells in order to perform the analysis.

### Step 3.1: Generate Cost Map
This step creates the mapping from object labels in your legend to the movement costs defined in the configuration.

In [5]:
with open(LEGEND_PATH, 'r', encoding='utf-8') as f:
    legend_data = json.load(f)

COST_MAP = generate_cost_map(legend_data, KEYWORD_COSTS)
print("\nGenerated Cost Map:")
print(COST_MAP)

🤖 Generating advanced cost map from keywords...
✅ Advanced cost map generated.

Generated Cost Map:
{'200 - on the floor': 1.0, '300 - on top of others': inf, '400 - attach to wall': inf, '500 - attach to ceiling': inf, 'Bar': inf, 'Barstool': inf, 'Bed Frame': 3.0, 'Bookcase / jewelry Armoire': inf, 'Bunk Bed': 3.0, 'Ceiling Lamp': 1.25, 'Chaise Longue Sofa': inf, 'Children Cabinet': inf, 'Classic Chinese Chair': inf, 'Coffee Table': inf, 'Corner/Side Table': inf, 'Couch Bed': 3.0, 'Desk': inf, 'Dining Chair': inf, 'Dining Table': inf, 'Double Bed': 3.0, 'Drawer Chest / Corner cabinet': inf, 'Dressing Chair': inf, 'Dressing Table': inf, 'Floor Lamp': 1.25, 'Folding chair': inf, 'Footstool / Sofastool / Bed End Stool / Stool': inf, 'Hanging Chair': inf, 'Kids Bed': 3.0, 'King-size Bed': 3.0, 'L-shaped Sofa': inf, 'Lazy Sofa': inf, 'Lounge Chair / Book-chair / Computer Chair': inf, 'Lounge Chair / Cafe Chair / Office Chair': inf, 'Loveseat Sofa': inf, 'Nightstand': inf, 'Pendant Lamp': 

### Step 3.2: Create Traversability Grid
Here, we convert the source image into a 2D grid where each cell's value represents its movement cost.

In [6]:
grid = create_traversability_grid(IMAGE_PATH, LEGEND_PATH, CELL_SIZE, COST_MAP)

▶️ Loading image 'reconstructed.png' and legend 'color_legend.json'...
ℹ️ Building color KD-Tree for fast nearest-neighbor lookup...
ℹ️ Mapping pixels to nearest legend color to determine costs...
ℹ️ Image dimensions: 1024x1024 pixels.
ℹ️ Grid cell size: 1x1 pixels.
ℹ️ Resulting graph dimensions: 1024x1024 nodes.
ℹ️ Downsampling high-resolution cost map to final grid...
✅ Cost-based traversability grid generated successfully.


### Step 3.3: Create NetworkX Graph
The cost grid is converted into a graph data structure, which is required for the A\* algorithm.

In [7]:
graph = create_networkx_graph(grid)

✅ NetworkX graph created with 299900 nodes and 1172298 edges.


### Step 3.4: Interactively Select Start & Goal Nodes
Running this cell will open a new window showing the cost grid.
- **First click**: Select the **START** node.
- **Second click**: Select the **GOAL** node.
The window will close automatically after you select the goal.

In [8]:
start_node, goal_node = get_interactive_nodes(grid)


--- Interactive Node Selection ---
A plot window has opened. Please click on the grid to select the START node.
📍 Start node selected: (267, 516)
Please click on the grid to select the GOAL node.
🏁 Goal node selected: (443, 655)


### Step 3.5: Run A* and Visualize Path
If start and goal nodes were selected, this cell runs the A\* algorithm and generates the final visualization comparing the original image with the cost grid and the calculated path.

In [9]:
if start_node and goal_node:
    # Set the backend back to one that can be displayed inline
    %matplotlib inline
    path = run_astar_pathfinding(graph, start_node, goal_node)
    visualize_results(IMAGE_PATH, grid, CELL_SIZE, path)
else:
    print("Skipping pathfinding and visualization because nodes were not selected.")


--- Running A* Pathfinding ---
📍 Start Node: (267, 516)
🏁 Goal Node:  (443, 655)
✅ Path found with 177 nodes and a total cost of 233.58.

💾 Visualization with path saved to 'reconstructed'


C:\Users\Hagai.LAPTOP-QAG9263N\AppData\Local\Temp\ipykernel_17180\1917352618.py:218: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


### Step 3.6: Save Artifacts
Finally, we save the generated grid and graph to files for later use or analysis.

In [10]:
# Re-enable interactive backend if you want to re-run the selection
%matplotlib tk

# --- Save outputs ---
output_stem = IMAGE_PATH.with_suffix('').stem.replace('_image', '')
output_dir = IMAGE_PATH.parent

# Save the cost grid as a NumPy array
grid_path = output_dir / f"{output_stem}_cost_grid.npy"
np.save(grid_path, grid)
print(f"💾 Cost grid saved to '{grid_path}'")

# Save the graph in GraphML format
graph_path = output_dir / f"{output_stem}_cost_graph.graphml"
nx.write_graphml(graph, graph_path)
print(f"💾 NetworkX graph saved to '{graph_path}'")

💾 Cost grid saved to 'test_images\reconstructed_cost_grid.npy'
💾 NetworkX graph saved to 'test_images\reconstructed_cost_graph.graphml'
